In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from datetime import date
pd.set_option('display.max_columns', 500)

In [ ]:
test_df = pd.read_csv("../data/boxer dc kzn/april 2018.csv")

In [ ]:
test_df[:5]

In [ ]:
test_df.describe()

### Data cleanup

In [ ]:
#drop all unnamed columns
print('Initial cols',list(test_df.columns.values))
print("")
unnamed_col_idxs = [idx for idx,col in enumerate(list(test_df.columns.values)) if "Unnamed" in col]
print('Unnamed cols indexes',unnamed_col_idxs)
print("")
test_df.drop(labels=test_df.columns[unnamed_col_idxs], axis='columns', inplace=True)
print('removed Unnamed cols',list(test_df.columns.values))


# Inspect the columns

In [ ]:
test_df[:5]

In [ ]:
test_df['LogOffReason'] = '/'+test_df['LogOffReason']


In [ ]:
test_df['LogOffReason']

In [ ]:
def column_to_flags(df, col, delimiter):
    # split col column into separate True/False columns for each found flag delimited by delimiter
    # assumes the entries in col are strings containing flags delimited by delimiter
    # e.g.: "flag0;flag1;flag2" (delimiter is ;)
    
    # preprocess column by :
    # converting nan to string
    df[col] = df[col].fillna('')
    #adding delimiter to start of entry, to support flags that are substrings of other flags
    df[col] = delimiter + df[col] 
    
    flags_strings = []
    #1) find all possible flags
    #1.1) extract each flag separated by delimiter from entries in col column
    for entry_str in df[col]:
        flags_strings += entry_str.split(delimiter)
    #1.2) get rid of redundand flags by making the list a set
    flags_set = set(flags_strings)
    flags_set.remove('') # remove empty entry    
    #1.3) convert to sorted list
    flags_list = sorted(list(flags_set))
    print('Converted ', col ,' column into flags',flags_list)
    #1.4) add 
    #1.5) add each flag as a new column and mark entry with True
    for flag_str in flags_list:
        df[flag_str] = [delimiter+flag_str+delimiter in entry_str for entry_str in df[col]]
    #1.6) remove original column
    df.drop(labels=col, axis='columns', inplace=True )    
    
    return df

In [ ]:
df = column_to_flags(df=test_df, col="LogOffReason", delimiter="/")

In [ ]:
df = column_to_flags(df=df, col="Result", delimiter="/")

### Per driver stats

In [ ]:
bymachine = test_df.groupby('Driver')

In [ ]:
# this value counts in fact orders the list by driver, from lowest to highest number or letter
bymachine['Harsh Acceleration'].value_counts()

### Per machine stats  

### explore date & time 

In [ ]:
df["Date & Time"][:5]

# Observe the problems with the date, where we have 2018/04/3011:45:47PM instead of 2018/04/30 11:45:47 PM
# so we split in the first 10 and the last 10

### Split Date & Time into Data and Time

In [ ]:
extract_date = lambda x: x[:10]
extract_time = lambda x: x[10:]
df['Date'] = df['Date & Time'].map(extract_date)
df['Time'] = df['Date & Time'].map(extract_time)

In [ ]:
# since we have a missing space sometimes, we check for the space
separate_date_time = lambda x: x[:10]+' '+x[10:] if x[10] != ' ' else x 
df["Date & Time"] = df["Date & Time"].map(separate_date_time)


In [ ]:
df["Date & Time"][:3]

In [ ]:
print(df['Date'].head())
print(df['Time'].head())

In [ ]:
# from fastai library:
# https://github.com/fastai/fastai/blob/master/fastai/structured.py
def add_datepart(df, fldname, drop=True, time=False):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre +'_' + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
add_datepart(df,'Date & Time',drop=False,time=True)

In [ ]:
df.columns.values

# Remove spaces and & from names

In [ ]:
df.columns = df.columns.str.strip().str.replace(' ', '_')
df.columns

### Reversed dataframe

In [ ]:
def reverse_dataframe(df):
    df_index_col = df.index
    reversed_df = df.iloc[::-1]
    reversed_df.index = df_index_col
    reversed_df.to_csv("reversed_df.csv")
    return reversed_df

In [ ]:
reversed_df = reverse_dataframe(df)

### If the driver shift is between 2 months

In [ ]:
def add_line_df(df):
    # if the first line does not contain Not Logged On or Driver change
    if(df.Driver[0] != 'Not Logged On' and df.Driver_Change[0] != True):
        df = pd.concat([df[:1], df], ignore_index=True)
        df.iloc[0, df.columns.get_loc('Driver')] = 'Not Logged On'
        df.iloc[0, df.columns.get_loc('Driver_Change')] = True
        # save
        df.to_csv("iverse_modif_df.csv")
        
    return df

In [ ]:
new_df = add_line_df(reversed_df)

# Get total driving time / driver

#### from the "Ignition on = True" after "Logged on" until "Ignition on = False" before "Logged off" 

(1) Get the ! values of driver

In [ ]:
# print the fields in Driver
print(new_df.Driver.values)
# get how many ! drivers are
count_unique_drivers = len(new_df.Driver.unique()) - 1
print("cout ! values: ",count_unique_drivers)
unique_drivers = new_df.Driver.unique()
print("unique_drivers: ",unique_drivers)

(1) (b) remove Not Logged On tag from unique_drivers

In [ ]:
# get the index of 'Not Logged On' element
index_nlo = np.where(unique_drivers=='Not Logged On')[0][0]
print("index_nlo: ",index_nlo)
unique_drivers = np.delete(unique_drivers, index_nlo)
print("unique drivers: ",unique_drivers)

(2) get teh indexes of "Not Logged On"

### Get the index of Driver Change

for idx, val in small_df.Driver_Change

In [ ]:
indexes = [idx for idx,col in enumerate(new_df.Driver.values) if col=='Not Logged On']

In [ ]:
indexes

# Get the driver logged on before this driver change

In [ ]:

def get_time_logged_on(df, indexes):
    # am index si indexul indexului
    for i, ind in enumerate(indexes):
        if(df.iloc[ind+1]['Driver'] != 'Not Logged On') and ind !=indexes[-1]:
            #print("i = {} and ind = {} ".format(i,ind))
            #print("ind+1 = ",ind+1)
            #print("indexes[i+1]-1 = ",indexes[i+1]-1)
            #print("i= {} and index={} ".format(i,ind))
            print("Driver: ",df.iloc[ind+1]['Driver'])
            print("Log on time: ",df.iloc[ind+1]['Date_&_Time'])
            print( "Log off time: ",df.iloc[indexes[i+1]]['Date_&_Time'])                                   
    
        elif(ind ==indexes[-1]):
            get_the_last_index_of_df = df.index[-1]
            print("Driver: ",df.iloc[ind+1]['Driver'])
            print("Log on time: ",df.iloc[ind+1]['Date_&_Time'])
            print( "Log off time: ",df.iloc[get_the_last_index_of_df]['Date_&_Time'])   
            

In [ ]:
#small_df = new_df[:66]
get_time_logged_on(new_df, indexes)